In [97]:
from seleniumbase import Driver
from selenium.webdriver.common.keys import Keys

from rich.pretty import pprint
import json
import re

In [98]:
driver = Driver(uc=True, uc_cdp_events=True)

raw = []

def save(data):
    req = {}
    req["id"] = data["params"]["requestId"]
    req["path"] = data["params"]["headers"][":path"]
    if "/getData/" in req["path"]:
        raw.append(req)

driver.add_cdp_listener('Network.requestWillBeSentExtraInfo', save)

driver.get(
    "https://www.marinetraffic.com/en/ais/home/centerx:132.2/centery:43.0/zoom:10")

driver.sleep(3)

driver.click('button:contains("AGREE")')

driver.sleep(1)

# Magic clicks
driver.click("div#map_canvas")
map = driver.find_element("div#map_canvas")
map.send_keys(Keys.RETURN)

# Zoom in
for i in range(3):
    driver.click('a[title="Zoom in"]')
    driver.sleep(1)

# Zoom out
for i in range(6):
    driver.click('a[title="Zoom out"]')
    driver.sleep(1)

driver.sleep(3)

for req in raw:
    try:
        response = driver.execute_cdp_cmd(
            "Network.getResponseBody", {"requestId": req["id"]})
        req["response"] = response
    except Exception as e:
        req["response"] = None

driver.sleep(3)
driver.close()

In [99]:
len(raw)

63

In [100]:
def shipRawParser(raw):
    regex = r"z:(?P<z>\d+)\/X:(?P<x>\d+)\/Y:(?P<y>\d+)"

    shipData = []
    for r in raw:
        if "get_data_json_4" in r["path"]:
            matches = re.findall(regex, r["path"])
            z = matches[0][0]
            x = matches[0][1]
            y = matches[0][2]

            rows = json.loads(r["response"]["body"])["data"]["rows"]
            for row in rows:
                data = {}
                data["z"] = z
                data["x"] = x
                data["y"] = y

                data["data"] = row

                shipData.append(data)

    return shipData

shipData = shipRawParser(raw)

In [101]:
len(shipData)

2096

In [102]:
pprint(shipData[:3])

[
│   {
│   │   'z': '10',
│   │   'x': '444',
│   │   'y': '187',
│   │   'data': {
│   │   │   'LAT': '43.12085',
│   │   │   'LON': '132.3367',
│   │   │   'SPEED': '22',
│   │   │   'COURSE': '192',
│   │   │   'HEADING': '38',
│   │   │   'ELAPSED': '1',
│   │   │   'DESTINATION': 'FOR ORDER',
│   │   │   'FLAG': 'RU',
│   │   │   'LENGTH': '130',
│   │   │   'ROT': '0',
│   │   │   'SHIPNAME': 'PLAVUCHIY KRAN 3500',
│   │   │   'SHIPTYPE': '3',
│   │   │   'SHIP_ID': '7203862',
│   │   │   'WIDTH': '48',
│   │   │   'L_FORE': '71',
│   │   │   'W_LEFT': '29',
│   │   │   'DWT': '15045',
│   │   │   'GT_SHIPTYPE': '127'
│   │   }
│   },
│   {
│   │   'z': '10',
│   │   'x': '444',
│   │   'y': '187',
│   │   'data': {
│   │   │   'LAT': '43.22013',
│   │   │   'LON': '132.3126',
│   │   │   'SPEED': '18',
│   │   │   'COURSE': '51',
│   │   │   'HEADING': '277',
│   │   │   'ELAPSED': '381',
│   │   │   'DESTINATION': 'CN ZOS>SUKHODOL',
│   │   │   'FLAG': 'PA',
│   │   │   'LENGTH': '182',
│   │   │   'ROT': '-33',
│   │   │   'SHIPNAME': 'CS FORTUNE',
│   │   │   'SHIPTYPE': '7',
│   │   │   'SHIP_ID': '463849',
│   │   │   'WIDTH': '28',
│   │   │   'L_FORE': '167',
│   │   │   'W_LEFT': '13',
│   │   │   'DWT': '34006',
│   │   │   'GT_SHIPTYPE': '9'
│   │   }
│   },
│   {
│   │   'z': '10',
│   │   'x': '444',
│   │   'y': '187',
│   │   'data': {
│   │   │   'LAT': '43.11924',
│   │   │   'LON': '132.3346',
│   │   │   'SPEED': '0',
│   │   │   'COURSE': '0',
│   │   │   'HEADING': None,
│   │   │   'ELAPSED': '528',
│   │   │   'DESTINATION': 'TEST',
│   │   │   'FLAG': 'RU',
│   │   │   'LENGTH': '257',
│   │   │   'SHIPNAME': 'VALENTIN PIKUL',
│   │   │   'SHIPTYPE': '8',
│   │   │   'SHIP_ID': '8649383',
│   │   │   'WIDTH': '34',
│   │   │   'L_FORE': '210',
│   │   │   'W_LEFT': '21'
│   │   }
│   }
]

In [103]:
def shipDataParser(shipData):
  ships = {}

  for data in shipData:
    ship = {} | data["data"]
    ship["TILE_X"] = data["x"]
    ship["TILE_Y"] = data["y"]
    ship["TILE_Z"] = data["z"]

    id = data["data"]["SHIP_ID"]

    # Keep ship with highest Z value
    if id in ships:
      if int(ship["TILE_Z"]) > int(ships[id]["TILE_Z"]):
        ships[id] = ship
    else:
      ships[id] = ship

  return ships

ships = shipDataParser(shipData)

In [104]:
len(ships)

954

In [105]:
with open('ships_example_data.json', 'w') as f:
    json.dump(ships, f)